In [2]:
from sympy import mod_inverse

# Given primes p and q
p = 115298249408953906298939189175783351549219173898770205207152008519638746932703
q = 106934579549662309243376191428977731334267873165644052245494135513086209528559

# Calculate the modulus
n = p * q

# Calculate Euler's totient function φ(n)
phi_n = (p - 1) * (q - 1)

# Given exponent e
e = 65537

# Calculate the multiplicative inverse of e modulo φ(n)
d = mod_inverse(e, phi_n)
if(e*d%phi_n==1):
    print("true")
else:
    print("false")
print("n=",n)
print("phi=",phi_n)
print("d=",d)

true
n= 12329369823358586822326848571537893133945999655484440067209664044969343984710078440644548204795633321028007822042159402545605555448031381247696832929564977
phi= 12329369823358586822326848571537893133945999655484440067209664044969343984709856207815589588580091005647403060959275915498541141190578735103664107973103716
d= 5261574610062910359748497808089656151032267549088130671219614630051766369598322450221804623547125825807506733736805603013003320823627966848944531665956097


In [25]:
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from Crypto.Util.Padding import pad
import binascii

# Constants provided by the user
x =111106725858912721126303010983641533711713
message = "Hello World!!!"

# Convert x from hex to decimal
#x = int(x_hex, 16)

# Encrypt x with RSA
y = pow(x, e, n)

# Hash x to get the AES key
hasher = SHA256.new()
hasher.update(x_hex.encode('utf-8'))  # The hasher expects bytes, so we encode the hex string
k = hasher.digest()  # This is a bytes object of the hash
#print(k)
# AES encryption with the given key k
cipher = AES.new(k, AES.MODE_CBC, iv=b'\x00'*16)  # The IV is 16 bytes of zeros
padded_message = pad(message.encode(), AES.block_size)  # The message must be padded to fit the block size
ciphertext = cipher.encrypt(padded_message)

# Convert the ciphertext to a hexadecimal representation
c_hex = binascii.hexlify(ciphertext).decode('utf-8')

# The final output is the pair (y, c), where y is the encrypted x, and c is the encrypted message.
# We output them as hexadecimal strings.
y_hex = hex(y)[2:]  # Convert the encrypted x to hex and strip the '0x' prefix
result = (y_hex, c_hex)

result


('18d874f6fccb2d6b4848325ddf203498ad16a1c4757735355ea2435cf1a2e222051e01c06d5400f44a25c136d8ffe1a1cc1ead860c0776e2a35732298e60e7eb276',
 '9b609eb23c29891ea34d5703c0266df9')

In [27]:
from Crypto.Util.number import long_to_bytes
from Crypto.Hash import SHA256
from Crypto.Cipher import AES
import binascii

# Given RSA private key components
n = 67638648536063270601672550371
d = 6685099266753336756135252851

# Hexadecimal representation of y
y_hex = "BC9D481E538964F414EB93C8"

# Convert hex to long int for RSA decryption
y_long = int(y_hex, 16)

# Decrypt y using RSA to get x
x_long = pow(y_long, d, n)

# Convert long int x back to bytes
x_bytes = long_to_bytes(x_long)

# Hash x to obtain the AES key
aes_key = SHA256.new(x_bytes).digest()

# Hexadecimal representation of the ciphertext c
c_hex = ("C16E4AEBC9D2027E6608805CD9EEA46E800C5"
         "E069822CD71D5718CC945FFBF2A7E36F5"
         "8A705D9720DB4203A2205A193920937BEA"
         "3288A34A8F857EE535674E7DDE452BB1456")

# Convert hex to bytes for AES decryption
c_bytes = binascii.unhexlify(c_hex)

# Initialize AES-CBC cipher with the derived key and an IV of all zeroes
cipher = AES.new(aes_key, AES.MODE_CBC, iv=long_to_bytes(0, 16))

# Decrypt the ciphertext to get the plaintext message m
m_bytes = cipher.decrypt(c_bytes)

# Attempt to manually unpad the message
def manual_unpad(data):
    pad_len = data[-1]
    if pad_len < 1 or pad_len > 16:
        raise ValueError("Invalid padding length.")
    if data.endswith(bytes([pad_len]) * pad_len):
        return data[:-pad_len]
    else:
        raise ValueError("Invalid padding.")

# Use the manual unpadding function instead of unpad_pkcs7
try:
    m_unpadded = manual_unpad(m_bytes)
    m_str = m_unpadded.decode('utf-8')
except ValueError as e:
    print("An error occurred during decryption or unpadding:", e)
    m_str = None  # In case of an error, do not use the m_bytes for output

if m_str:
    print("Decrypted message m:", m_str)
else:
    print("Decrypted message m could not be determined.")

print("Decrypted integer x in decimal:", x_long)


Error: Odd-length string

In [13]:
import hashlib
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Util.Padding import pad
import binascii

# The message to encrypt
message = b"Hello World!!!"

# Step 1: Random number x (given in hex, convert to int)
x_hex = '1468369091346689468057626077110561'
x = int(x_hex, 16)

# Step 2: RSA encryption with the correct y value provided by the user
y_correct = 8441337013066137990163781585057952690193509976005942098377220241417205509147148382667275889497884138355302679507679060946846382921432798426601428824799325
y_hex_correct = hex(y_correct)[2:]  # Convert to hexadecimal

# Step 3: Hash x to get the AES key
x_bytes = x.to_bytes((x.bit_length() + 7) // 8, byteorder='big')
k = hashlib.sha256(x_bytes).digest()

# Step 4: AES encryption of the message
# For simplicity, we'll use a zero IV
iv = b'\x00' * AES.block_size
cipher_aes = AES.new(k, AES.MODE_CBC, iv)
ciphertext_aes = cipher_aes.encrypt(pad(message, AES.block_size))

# Convert ciphertext to hex
c_hex = binascii.hexlify(ciphertext_aes).decode()

# Step 5: Output the resulting ciphertext (y, c)
final_ciphertext_corrected = (y_hex_correct, c_hex)

final_ciphertext_corrected


('a12c6555895811fcf162d368d60c22e3a094ef3ed5f072eb1952383ec84eae0418c96fef2110698a2bfd940e4a359f84f70195b64c7a37cca48a3fc6a27eb85d',
 '03203f38af1390b8b4a961f9c8eaabe8')